#Objetivo do MVP
Investigar os principais fatores que influenciam a satisfação dos clientes no e-commerce brasileiro, com base nas avaliações de pedidos da Olist.

O objetivo é entender como variáveis como tempo de entrega, categoria do produto, localização do vendedor, valor da compra e método de pagamento afetam a experiência do consumidor e sua avaliação do pedido.

##Perguntas de negócio:
1. Qual o perfil dos pedidos com melhores e piores avaliações?

2. O tempo de entrega impacta na nota da avaliação?

3. Há categorias de produtos com maiores taxas de insatisfação?

4. Compras com valor mais alto tendem a ter melhores avaliações?

5. Existem estados ou regiões com mais reclamações?

6. A forma de pagamento influencia a nota dada pelo cliente?

##Camada Bronze

In [0]:
# Diretório base do projeto (no DBFS)
bronze_path = "/mnt/mvp_olist/bronze/"

In [0]:
# Leitura do CSV orders
df_orders = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_orders_dataset.csv", header=True, inferSchema=True)

# Escrita no formato Delta (camada Bronze)
df_orders.write.format("delta").mode("overwrite").save(bronze_path + "orders")


In [0]:
df_customers = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_customers_dataset.csv", header=True, inferSchema=True)
df_customers.write.format("delta").mode("overwrite").save(bronze_path + "customers")


In [0]:
df_items = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_order_items_dataset.csv", header=True, inferSchema=True)
df_items.write.format("delta").mode("overwrite").save(bronze_path + "order_items")


In [0]:
df_payments = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_order_payments_dataset.csv", header=True, inferSchema=True)
df_payments.write.format("delta").mode("overwrite").save(bronze_path + "order_payments")


In [0]:
df_reviews = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
df_reviews.write.format("delta").mode("overwrite").save(bronze_path + "order_reviews")


In [0]:
df_products = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_products_dataset.csv", header=True, inferSchema=True)
df_products.write.format("delta").mode("overwrite").save(bronze_path + "products")


In [0]:
df_sellers = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_sellers_dataset.csv", header=True, inferSchema=True)
df_sellers.write.format("delta").mode("overwrite").save(bronze_path + "sellers")


In [0]:
df_geo = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/olist_geolocation_dataset.csv", header=True, inferSchema=True)
df_geo.write.format("delta").mode("overwrite").save(bronze_path + "geolocation")


In [0]:
df_cat_translation = spark.read.csv("/FileStore/shared_uploads/martinsnunes.nati@gmail.com/product_category_name_translation.csv", header=True, inferSchema=True)
df_cat_translation.write.format("delta").mode("overwrite").save(bronze_path + "product_category_name_translation")


In [0]:
# Validação da camada bronze
df_orders.show(5)


+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

##Camada Silver

In [0]:
silver_path = "/mnt/mvp_olist/silver/"

In [0]:
# Leitura dos dados da camada Bronze
df_orders = spark.read.format("delta").load(bronze_path + "orders")
df_items = spark.read.format("delta").load(bronze_path + "order_items")

# Importar funções úteis
from pyspark.sql.functions import to_timestamp

# Tratamento de datas
df_orders = df_orders.withColumn("order_purchase_timestamp", to_timestamp("order_purchase_timestamp")) \
                     .withColumn("order_approved_at", to_timestamp("order_approved_at")) \
                     .withColumn("order_delivered_carrier_date", to_timestamp("order_delivered_carrier_date")) \
                     .withColumn("order_delivered_customer_date", to_timestamp("order_delivered_customer_date")) \
                     .withColumn("order_estimated_delivery_date", to_timestamp("order_estimated_delivery_date"))

# Join entre pedidos e itens
df_orders_items = df_orders.join(df_items, on="order_id", how="inner")

# Salvando camada Silver
df_orders_items.write.format("delta").mode("overwrite").save(silver_path + "orders_items")


In [0]:
df_customers = spark.read.format("delta").load(bronze_path + "customers")

# Selecionar colunas relevantes
df_customers = df_customers.select("customer_id", "customer_city", "customer_state")

df_customers.write.format("delta").mode("overwrite").save(silver_path + "customers")


In [0]:
df_products = spark.read.format("delta").load(bronze_path + "products")
df_cat = spark.read.format("delta").load(bronze_path + "product_category_name_translation")

df_products = df_products.join(df_cat, on="product_category_name", how="left")

df_products.write.format("delta").mode("overwrite").save(silver_path + "products")


In [0]:
from pyspark.sql.functions import col

df_payments = spark.read.format("delta").load(bronze_path + "order_payments")

df_payments = df_payments.withColumn("payment_value", col("payment_value").cast("double"))

df_payments.write.format("delta").mode("overwrite").save(silver_path + "order_payments")


In [0]:
df_reviews = spark.read.format("delta").load(bronze_path + "order_reviews")

# Converter a data para timestamp
df_reviews = df_reviews.withColumn("review_creation_date", to_timestamp("review_creation_date")) \
                       .withColumn("review_answer_timestamp", to_timestamp("review_answer_timestamp"))

df_reviews.write.format("delta").mode("overwrite").save(silver_path + "order_reviews")


In [0]:
df_sellers = spark.read.format("delta").load(bronze_path + "sellers")
df_sellers.write.format("delta").mode("overwrite").save(silver_path + "sellers")


In [0]:
df_geo = spark.read.format("delta").load(bronze_path + "geolocation")
df_geo.write.format("delta").mode("overwrite").save(silver_path + "geolocation")


##Camada Gold

In [0]:
silver_path = "/mnt/mvp_olist/silver/"

df_orders_items = spark.read.format("delta").load(silver_path + "orders_items")
df_reviews = spark.read.format("delta").load(silver_path + "order_reviews")
df_customers = spark.read.format("delta").load(silver_path + "customers")
df_products = spark.read.format("delta").load(silver_path + "products")
df_payments = spark.read.format("delta").load(silver_path + "order_payments")


In [0]:
# Imports úteis
from pyspark.sql.functions import col, datediff

# Join com reviews
df = df_orders_items.join(df_reviews.select("order_id", "review_score"), on="order_id", how="left")

# Join com clientes
df = df.join(df_customers, on="customer_id", how="left")

# Join com produtos
df = df.join(df_products.select("product_id", "product_category_name_english"), on="product_id", how="left")

# Join com pagamentos
df = df.join(df_payments.select("order_id", "payment_type", "payment_value"), on="order_id", how="left")

# Calcular tempo de entrega (em dias)
df = df.withColumn("delivery_days", datediff("order_delivered_customer_date", "order_purchase_timestamp"))


In [0]:
gold_path = "/mnt/mvp_olist/gold/"

df.write.format("delta").mode("overwrite").saveAsTable("gold_orders_analytics")

In [0]:
%sql
-- Validação camada Gold

SELECT * FROM gold_orders_analytics LIMIT 10;

order_id,product_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value,review_score,customer_city,customer_state,product_category_name_english,payment_type,payment_value,delivery_days
e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33.000+0000,2017-10-02T11:07:15.000+0000,2017-10-04T19:55:00.000+0000,2017-10-10T21:25:13.000+0000,2017-10-18T00:00:00.000+0000,1,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06T11:07:15.000+0000,29.99,8.72,4,sao paulo,SP,housewares,voucher,18.59,8
e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33.000+0000,2017-10-02T11:07:15.000+0000,2017-10-04T19:55:00.000+0000,2017-10-10T21:25:13.000+0000,2017-10-18T00:00:00.000+0000,1,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06T11:07:15.000+0000,29.99,8.72,4,sao paulo,SP,housewares,voucher,2.0,8
e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33.000+0000,2017-10-02T11:07:15.000+0000,2017-10-04T19:55:00.000+0000,2017-10-10T21:25:13.000+0000,2017-10-18T00:00:00.000+0000,1,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06T11:07:15.000+0000,29.99,8.72,4,sao paulo,SP,housewares,credit_card,18.12,8
53cdb2fc8bc7dce0b6741e2150273451,595fac2a385ac33a80bd5114aec74eb8,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37.000+0000,2018-07-26T03:24:27.000+0000,2018-07-26T14:31:00.000+0000,2018-08-07T15:27:45.000+0000,2018-08-13T00:00:00.000+0000,1,289cdb325fb7e7f891c38608bf9e0962,2018-07-30T03:24:27.000+0000,118.7,22.76,4,barreiras,BA,perfumery,boleto,141.46,14
47770eb9100c2d0c44946d9cf07ec65d,aa4383b373c6aca5d8797843e5594415,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49.000+0000,2018-08-08T08:55:23.000+0000,2018-08-08T13:50:00.000+0000,2018-08-17T18:06:29.000+0000,2018-09-04T00:00:00.000+0000,1,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13T08:55:23.000+0000,159.9,19.22,5,vianopolis,GO,auto,credit_card,179.12,9
949d5b44dbf5de918fe9c16f97b45f8a,d0b61bfb1de832b15ba9d266ca96e5b0,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06.000+0000,2017-11-18T19:45:59.000+0000,2017-11-22T13:39:59.000+0000,2017-12-02T00:28:42.000+0000,2017-12-15T00:00:00.000+0000,1,66922902710d126a0e7d26b0e3805106,2017-11-23T19:45:59.000+0000,45.0,27.2,5,sao goncalo do amarante,RN,pet_shop,credit_card,72.2,14
ad21c59c0840e6cb83a9ceb5573f8159,65266b2da20d04dbe00c5c2d3bb7859e,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39.000+0000,2018-02-13T22:20:29.000+0000,2018-02-14T19:46:34.000+0000,2018-02-16T18:17:02.000+0000,2018-02-26T00:00:00.000+0000,1,2c9e548be18521d1c43cde1c582c6de8,2018-02-19T20:31:37.000+0000,19.9,8.72,5,santo andre,SP,stationery,credit_card,28.62,3
a4591c265e18cb1dcee52889e2d8acc3,060cb19345d90064d1015407193c233d,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09T21:57:05.000+0000,2017-07-09T22:10:13.000+0000,2017-07-11T14:58:04.000+0000,2017-07-26T10:57:55.000+0000,2017-08-01T00:00:00.000+0000,1,8581055ce74af1daba164fdbd55a40de,2017-07-13T22:10:13.000+0000,147.9,27.36,4,congonhinhas,PR,auto,credit_card,175.26,17
136cce7faa42fdb2cefd53fdc79a6098,a1804276d9941ac0733cfd409f5206eb,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11T12:22:08.000+0000,2017-04-13T13:25:17.000+0000,null,null,2017-05-09T00:00:00.000+0000,1,dc8798cbf453b7e0f98745e396cc5616,2017-04-19T13:25:17.000+0000,49.9,16.05,2,santa rosa,RS,null,credit_card,65.95,null
6514b8ad8028c9f2cc2374ded245783f,4520766ec412348b8d4caa5e8a18c464,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16T13:10:30.000+0000,2017-05-16T13:22:11.000+0000,2017-05-22T10:07:46.000+0000,2017-05-26T12:55:51.000+0000,2017-06-07T00:00:00.000+0000,1,16090f2ca825584b5a147ab24aa30c86,2017-05-22T13:22:11.000+0000,59.99,15

# Análises

## Pergunta 1
Qual o perfil dos pedidos com melhores e piores avaliações?

In [0]:
%sql
-- Distribuição geral das notas
SELECT 
  review_score, 
  COUNT(*) AS total
FROM gold_orders_analytics
GROUP BY review_score
ORDER BY review_score

review_score,total
null,978
1,14857
2,4085
3,9840
4,22286
5,66264


In [0]:
%sql
-- Perfil por forma de pagamento
-- Nota 1
SELECT 
  payment_type, 
  COUNT(*) AS nota_1
FROM gold_orders_analytics
WHERE review_score = 1
GROUP BY payment_type
ORDER BY nota_1 DESC

payment_type,nota_1
credit_card,10984
boleto,2897
voucher,796
debit_card,177
null,3


In [0]:
%sql
-- Nota 5
SELECT 
  payment_type, 
  COUNT(*) AS nota_5
FROM gold_orders_analytics
WHERE review_score = 5
GROUP BY payment_type
ORDER BY nota_5 DESC

payment_type,nota_5
credit_card,48968
boleto,12773
voucher,3494
debit_card,1029


In [0]:
%sql
-- Perfil por categoria de produto
-- Nota 1
SELECT 
  product_category_name_english, 
  COUNT(*) AS nota_1
FROM gold_orders_analytics
WHERE review_score = 1
GROUP BY product_category_name_english
ORDER BY nota_1 DESC
LIMIT 10

product_category_name_english,nota_1
bed_bath_table,1724
furniture_decor,1303
computers_accessories,1199
health_beauty,1131
sports_leisure,1061
housewares,869
watches_gifts,780
garden_tools,602
telephony,600
auto,512


In [0]:
%sql
-- Nota 5
SELECT 
  product_category_name_english, 
  COUNT(*) AS nota_5
FROM gold_orders_analytics
WHERE review_score = 5
GROUP BY product_category_name_english
ORDER BY nota_5 DESC
LIMIT 10

product_category_name_english,nota_5
bed_bath_table,6166
health_beauty,6024
sports_leisure,5284
furniture_decor,4698
computers_accessories,4347
housewares,4203
watches_gifts,3450
toys,2622
garden_tools,2567
auto,2454


In [0]:
%sql
-- Perfil por estado do cliente
-- Nota 1
SELECT 
  customer_state, 
  COUNT(*) AS nota_1
FROM gold_orders_analytics
WHERE review_score = 1
GROUP BY customer_state
ORDER BY nota_1 DESC

customer_state,nota_1
SP,5268
RJ,2802
MG,1557
RS,816
PR,658
BA,626
SC,556
DF,332
ES,312
GO,303


In [0]:
%sql
-- Nota 5 
SELECT 
  customer_state, 
  COUNT(*) AS nota_5
FROM gold_orders_analytics
WHERE review_score = 5
GROUP BY customer_state
ORDER BY nota_5 DESC

customer_state,nota_5
SP,29267
RJ,8040
MG,7867
RS,3650
PR,3484
SC,2372
BA,1932
DF,1370
GO,1290
ES,1248


## Pergunta 2
O tempo de entrega impacta na nota da avaliação?


In [0]:
%sql
-- Nota média por tempo de entrega (em dias)
SELECT 
  delivery_days, 
  ROUND(AVG(review_score), 2) AS avg_review_score,
  COUNT(*) AS total_orders
FROM gold_orders_analytics
WHERE delivery_days IS NOT NULL
GROUP BY delivery_days
ORDER BY delivery_days


delivery_days,avg_review_score,total_orders
0,5.0,1
1,4.41,765
2,4.43,3467
3,4.36,4157
4,4.38,5385
5,4.3,6280
6,4.3,7924
7,4.3,8914
8,4.27,8587
9,4.24,7463


In [0]:
# Como o SQL puro não permite criar colunas condicionais com facilidade, fazemos a categorização no PySpark e depois salvamos uma nova tabela Gold, com a coluna entrega_categoria:

from pyspark.sql.functions import when, col

df_gold = spark.table("gold_orders_analytics")

df_gold = df_gold.withColumn("entrega_categoria", 
    when(col("delivery_days") <= 5, "Rápida (0-5)")
   .when(col("delivery_days") <= 10, "Média (6-10)")
   .when(col("delivery_days") <= 20, "Lenta (11-20)")
   .otherwise("Muito lenta (>20)")
)

df_gold.write.format("delta").mode("overwrite").saveAsTable("gold_orders_entrega_categoria")


In [0]:
%sql
-- Nota média por faixa de entrega
SELECT 
  entrega_categoria, 
  ROUND(AVG(review_score), 2) AS avg_review_score,
  COUNT(*) AS total_orders
FROM gold_orders_entrega_categoria
GROUP BY entrega_categoria
ORDER BY entrega_categoria



entrega_categoria,avg_review_score,total_orders
Lenta (11-20),4.12,40321
Muito lenta (>20),2.94,18024
Média (6-10),4.26,39910
Rápida (0-5),4.36,20055


##Pergunta 3
Há categorias de produtos com maiores taxas de insatisfação?

In [0]:
%sql
-- Criar uma tabela auxiliar com o total de pedidos por categoria
CREATE OR REPLACE TEMP VIEW total_por_categoria AS
SELECT 
  product_category_name_english AS categoria,
  COUNT(*) AS total_pedidos
FROM gold_orders_analytics
WHERE product_category_name_english IS NOT NULL
GROUP BY product_category_name_english


In [0]:
%sql
-- Criar outra view com os pedidos nota 1 por categoria

CREATE OR REPLACE TEMP VIEW insatisfacao_por_categoria AS
SELECT 
  product_category_name_english AS categoria,
  COUNT(*) AS total_nota_1
FROM gold_orders_analytics
WHERE review_score = 1
AND product_category_name_english IS NOT NULL
GROUP BY product_category_name_english


In [0]:
%sql
SELECT 
  t.categoria,
  COALESCE(i.total_nota_1, 0) AS nota_1,
  t.total_pedidos,
  ROUND(COALESCE(i.total_nota_1, 0) / t.total_pedidos, 3) AS taxa_insatisfacao
FROM total_por_categoria t
LEFT JOIN insatisfacao_por_categoria i
  ON t.categoria = i.categoria
ORDER BY taxa_insatisfacao DESC
LIMIT 10


categoria,nota_1,total_pedidos,taxa_insatisfacao
security_and_services,1,2,0.5
fashion_male_clothing,40,145,0.276
diapers_and_hygiene,10,39,0.256
furniture_bedroom,26,126,0.206
office_furniture,360,1788,0.201
fixed_telephony,53,272,0.195
home_comfort_2,6,31,0.194
la_cuisine,3,16,0.188
construction_tools_safety,35,199,0.176
party_supplies,8,46,0.174


## Pergunta 4
Compras com valor mais alto tendem a ter melhores avaliações?

In [0]:
from pyspark.sql.functions import when, col

df_valor = spark.table("gold_orders_analytics")

df_valor = df_valor.withColumn("faixa_valor",
    when(col("payment_value") < 50, "Abaixo de R$50")
   .when(col("payment_value") < 100, "R$50 a R$99")
   .when(col("payment_value") < 200, "R$100 a R$199")
   .when(col("payment_value") < 500, "R$200 a R$499")
   .when(col("payment_value") < 1000, "R$500 a R$999")
   .otherwise("Acima de R$1000")
)

# Salvar como nova tabela
df_valor.write.format("delta").mode("overwrite").saveAsTable("gold_orders_faixa_valor")


In [0]:
%sql
SELECT 
  faixa_valor,
  ROUND(AVG(review_score), 2) AS avg_review,
  COUNT(*) AS total_pedidos
FROM gold_orders_faixa_valor
GROUP BY faixa_valor
ORDER BY faixa_valor

faixa_valor,avg_review,total_pedidos
Abaixo de R$50,4.16,21684
Acima de R$1000,3.52,1760
R$100 a R$199,4.03,36696
R$200 a R$499,3.87,20921
R$50 a R$99,4.12,32829
R$500 a R$999,3.75,4420


## Pergunta 5
Existem estados ou regiões com mais reclamações?

In [0]:
%sql
-- Criar view com total de pedidos por estado
CREATE OR REPLACE TEMP VIEW total_por_estado AS
SELECT 
  customer_state,
  COUNT(*) AS total_pedidos
FROM gold_orders_analytics
GROUP BY customer_state


In [0]:
%sql
-- Criar view com total de notas 1 por estado
CREATE OR REPLACE TEMP VIEW insatisfacao_por_estado AS
SELECT 
  customer_state,
  COUNT(*) AS total_nota_1
FROM gold_orders_analytics
WHERE review_score = 1
GROUP BY customer_state


In [0]:
%sql
-- calcular a taxa de insatisfação
SELECT 
  t.customer_state,
  COALESCE(i.total_nota_1, 0) AS nota_1,
  t.total_pedidos,
  ROUND(COALESCE(i.total_nota_1, 0) / t.total_pedidos, 3) AS taxa_insatisfacao
FROM total_por_estado t
LEFT JOIN insatisfacao_por_estado i
  ON t.customer_state = i.customer_state
ORDER BY taxa_insatisfacao DESC
LIMIT 10


customer_state,nota_1,total_pedidos,taxa_insatisfacao
RR,11,52,0.212
MA,165,849,0.194
AL,85,462,0.184
RJ,2802,15425,0.182
SE,70,397,0.176
PA,186,1123,0.166
CE,254,1556,0.163
PI,90,574,0.157
BA,626,4068,0.154
MT,165,1128,0.146


## Pergunta 6
A forma de pagamento influencia a nota dada pelo cliente?

In [0]:
%sql
-- Nota média por forma de pagamento
SELECT 
  payment_type,
  ROUND(AVG(review_score), 2) AS avg_review_score,
  COUNT(*) AS total_pedidos
FROM gold_orders_analytics
GROUP BY payment_type
ORDER BY avg_review_score DESC


payment_type,avg_review_score,total_pedidos
debit_card,4.16,1699
credit_card,4.03,87258
boleto,4.02,23018
voucher,4.0,6332
null,1.0,3
